In [1]:
from ioi_utils import *
from sae_variants import *
from training import *
from mandala._next.imports import *
from mandala._next.common_imports import *

In [2]:
model = get_model()

/home/amakelov/workspace/current/conda_envs/serimats/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [8]:
model.W_E.shape

torch.Size([50257, 768])

In [9]:
vocab = model.to_str_tokens(input=torch.arange(50257).long())

In [15]:
doubledigit_indices = {}
def is_int(x: str): return x.isdigit()
for i, elt in enumerate(vocab):
    if len(elt) == 2 and elt[0].isdigit() and elt[1].isdigit():
        doubledigit_indices[elt] = i

In [16]:
doubledigit_indices

{'00': 405,
 '01': 486,
 '10': 940,
 '12': 1065,
 '50': 1120,
 '19': 1129,
 '11': 1157,
 '20': 1238,
 '30': 1270,
 '15': 1314,
 '14': 1415,
 '16': 1433,
 '13': 1485,
 '25': 1495,
 '18': 1507,
 '17': 1558,
 '24': 1731,
 '80': 1795,
 '40': 1821,
 '22': 1828,
 '60': 1899,
 '23': 1954,
 '29': 1959,
 '27': 1983,
 '26': 2075,
 '28': 2078,
 '99': 2079,
 '33': 2091,
 '70': 2154,
 '45': 2231,
 '35': 2327,
 '64': 2414,
 '75': 2425,
 '21': 2481,
 '38': 2548,
 '44': 2598,
 '36': 2623,
 '32': 2624,
 '39': 2670,
 '34': 2682,
 '05': 2713,
 '37': 2718,
 '48': 2780,
 '66': 2791,
 '55': 2816,
 '47': 2857,
 '08': 2919,
 '49': 2920,
 '09': 2931,
 '65': 2996,
 '07': 2998,
 '02': 2999,
 '04': 3023,
 '03': 3070,
 '68': 3104,
 '31': 3132,
 '67': 3134,
 '59': 3270,
 '06': 3312,
 '77': 3324,
 '58': 3365,
 '69': 3388,
 '88': 3459,
 '46': 3510,
 '57': 3553,
 '43': 3559,
 '42': 3682,
 '78': 3695,
 '79': 3720,
 '90': 3829,
 '95': 3865,
 '41': 3901,
 '56': 3980,
 '54': 4051,
 '98': 4089,
 '76': 4304,
 '52': 4309,
 '

In [18]:
torch.cosine_similarity(model.W_E[doubledigit_indices['10']], model.W_E[doubledigit_indices['47']], dim=0)

tensor(0.4514, device='cuda:0')